## Sentiment140 ✨ Logistic Regression

### Dataset

Load `sentiment140` dataset, assign column names and extract a sample (100.000)

In [1]:
import numpy as np
import pandas as pd
import random

# Load dataset
df = pd.read_csv('data/sentiment140.csv', header=None, usecols=[0, 5]) # no header in .csv
df.columns = ['Sentiment', 'Text']

# Sample
sample_size = 100_000
sample_indices = random.sample(range(len(df)), sample_size)
df = df.iloc[sample_indices]

display(df.head())

,Sentiment,Text
496790,0,@Franmedi mine doesn't either
954993,4,I had two people from Norway replying to my tw...
498852,0,theres this guy i know who only comes @ when h...
1445234,4,goodnight... and goodmorning
454977,0,@MrBrettYoung make sure to tweet on how awesom...


### Preprosessing

Convert to lowercase, `remove punctuation`, numbers and whitespaces.

In [2]:
df['Text'] = df['Text'].str.lower()                             # Convert text to lowercase
df['Text'] = df['Text'].str.replace('[^\w\s]', '', regex=True)  # Remove punctuation
df['Text'] = df['Text'].str.replace('\d+', '', regex=True)      # Remove numbers
df['Text'] = df['Text'].str.strip()                             # Strip whitespaces
display(df.head())

,Sentiment,Text
496790,0,franmedi mine doesnt either
954993,4,i had two people from norway replying to my tw...
498852,0,theres this guy i know who only comes when he...
1445234,4,goodnight and goodmorning
454977,0,mrbrettyoung make sure to tweet on how awesome...


### Training
Split data into train and test, convert to `numerical features`, fit the model (Logistic Regression).



In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Split the dataset into training and testing sets
X = df['Text']
y = df['Sentiment']
X1, X2, y1, y2 = train_test_split(X, y, test_size=0.2, random_state=0)

# Convert text into numerical features
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
X1 = tfidf_vectorizer.fit_transform(X1)
X2 = tfidf_vectorizer.transform(X2)

# Train the model
classifier = LogisticRegression(max_iter=1000)
classifier.fit(X1, y1)

# Make predictions on the test set
y_pred = classifier.predict(X2)

print("Sentiment140 samples =", sample_size)
print("Score on Train:", classifier.score(X1, y1).round(2))
print("Score on Test:", classifier.score(X2, y2).round(2))
print("Report:", classification_report(y2, y_pred))

Sentiment140 samples = 100000
Score on Train: 0.84
Score on Test: 0.75
Report:               precision    recall  f1-score   support

           0       0.76      0.74      0.75     10029
           4       0.75      0.77      0.76      9971

    accuracy                           0.75     20000
   macro avg       0.75      0.75      0.75     20000
weighted avg       0.75      0.75      0.75     20000



### Predictions

Predict unknow `reviews`, from reviews.csv file.

In [4]:
# Load reviews
df_reviews = pd.read_csv('data/reviews.csv')

# Convert the sentiment labels
df_reviews['Sentiment'] = df_reviews['Sentiment'].map({'negative': 0, 'positive': 4})

X_unknown = df_reviews['Review']
y_unknown = df_reviews['Sentiment']

# Convert text into numerical features
X_unknown = tfidf_vectorizer.transform(X_unknown)

# Make predictions on unknown set
y_unknown_pred = classifier.predict(X_unknown)
score = classifier.score(X_unknown, y_unknown)

print("Reviews (unknown):")
print("Unknown:\t", y_unknown.values)
print("Prediction:\t", y_unknown_pred)
print("Score on Unknown:", score.round(2))
print("Report:", classification_report(y_unknown, y_unknown_pred), "\n")

Reviews (unknown):
Unknown:	 [0 0 4 4 4 4 4 4 4 4 4 4 4 0 0 0 4 4 4 4 4 4 0 0 0 0 0 0 0 0 0 0 4 4 0]
Prediction:	 [4 0 4 4 4 0 4 4 4 4 4 4 4 4 0 4 4 4 4 4 4 4 4 4 0 4 0 4 4 0 0 4 4 4 0]
Score on Unknown: 0.71
Report:               precision    recall  f1-score   support

           0       0.88      0.44      0.58        16
           4       0.67      0.95      0.78        19

    accuracy                           0.71        35
   macro avg       0.77      0.69      0.68        35
weighted avg       0.76      0.71      0.69        35
 



### Random One

Prediction for `one random` review.

In [5]:
random_index = df_reviews.sample().index.item()

X_unknown_one = [df_reviews['Review'][random_index]]
y_unknown_one = df_reviews['Sentiment'][random_index]

labels = {0: 'negative', 4: 'positive'}

print("Review:\n", X_unknown_one[0])
print("Expected:", labels[y_unknown_one])

# Convert using vectorizer
X_unknown_one = tfidf_vectorizer.transform(X_unknown_one)
y_pred_one = classifier.predict(X_unknown_one)

print("Prediction:", labels[y_unknown_pred[0]])

Review:
 A very easy-to-read but comprehensive guide for ML, strongly recommend this to anyone who are interested in entering this field.
Expected: positive
Prediction: positive
